In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [30]:
!pip install optuna


   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   -------------------- ------------------- 1/2 [optuna]
   ---------------------------------------- 2/2 [optuna]



In [31]:
import optuna

In [32]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder 

In [33]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  heart_rate                

In [36]:
train.describe()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
count,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000
mean,349999.500000,50.359734,2.072411,80.230803,5.963695,7.002200,6.012733,25.874684,0.858766,116.294193,75.440924,70.167749,186.818801,53.823214,102.905854,123.081850,0.149401,0.181990,0.030324,0.623296
std,202072.738554,11.655520,1.048189,51.195071,1.463336,0.901907,2.022707,2.860705,0.037980,11.010390,6.825775,6.938722,16.730832,8.266545,19.022416,24.739397,0.356484,0.385837,0.171478,0.484560
min,0.000000,19.000000,1.000000,1.000000,0.100000,3.100000,0.600000,15.100000,0.680000,91.000000,51.000000,42.000000,117.000000,21.000000,51.000000,31.000000,0.000000,0.000000,0.000000,0.000000
25%,174999.750000,42.000000,1.000000,49.000000,5.000000,6.400000,4.600000,23.900000,0.830000,108.000000,71.000000,65.000000,175.000000,48.000000,89.000000,106.000000,0.000000,0.000000,0.000000,0.000000
50%,349999.500000,50.000000,2.000000,71.000000,6.000000,7.000000,6.000000,25.900000,0.860000,116.000000,75.000000,70.000000,187.000000,54.000000,103.000000,123.000000,0.000000,0.000000,0.000000,1.000000
75%,524999.250000,58.000000,3.000000,96.000000,7.000000,7.600000,7.400000,27.800000,0.880000,124.000000,80.000000,75.000000,199.000000,59.000000,116.000000,139.000000,0.000000,0.000000,0.000000,1.000000
max,699999.000000,89.000000,9.000000,747.000000,9.900000,9.900000,16.500000,38.400000,1.050000,163.000000,104.000000,101.000000,289.000000,90.000000,205.000000,290.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
test.isna().sum()

id                                    0
age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0


In [38]:
encode = LabelEncoder()

In [39]:
cat_cols = train.select_dtypes(include=['object']).columns

for col in cat_cols : 
    train[col]=encode.fit_transform(train[col])
    test[col]=encode.fit_transform(test[col])

In [40]:
train.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,0,2,1,2,0,0,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,0,4,1,4,2,0,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,1,2,1,2,2,1,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,0,4,1,2,0,0,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,1,4,1,4,2,1,0,1,0,1.0


In [41]:
x = train.drop(['id','diagnosed_diabetes'],axis=1)
y = train['diagnosed_diabetes']

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=2,test_size=0.2)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

models = {
    "Logistic": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(
        objective='binary:logistic',
        eval_metric='mlogloss'
    )
}


In [44]:
from sklearn.metrics import roc_auc_score

for name, model in models.items():
    model.fit(x_train, y_train)
    y_proba = model.predict_proba(x_test)
    
    # Extract only the probability for the positive class (class 1)
    # predict_proba returns probabilities for all classes, we need only the second column
    y_proba_positive = y_proba[:, 1]

    roc = roc_auc_score(
        y_test,
        y_proba_positive  # Use only probabilities for the positive class
    )

    print(f"{name}: ROC-AUC = {roc:.4f}")

Logistic: ROC-AUC = 0.6945
RandomForest: ROC-AUC = 0.6944
XGBoost: ROC-AUC = 0.7230


In [51]:
import xgboost as xgb
def objective(trial):
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
    }

    model = xgb.XGBClassifier(
        **param,
    )

    score = cross_val_score(model, x_test, y_test, cv=3, scoring="roc_auc").mean()
    return score

In [52]:


# Create and run the study with error handling
try:
    study = optuna.create_study(study_name="xgboost_study_cpu", direction="maximize")
    
    # Try with n_jobs=1 first to avoid potential multiprocessing issues
    study.optimize(objective, n_trials=100, show_progress_bar=True, n_jobs=1)
    
    # Check if we have completed trials before accessing best_params
    if len(study.trials) > 0:
        best_params = study.best_params
        print(f"\nBest parameters: {best_params}")
    else:
        print("No trials were completed. Check your objective function.")
except Exception as e:
    print(f"Optimization failed with error: {e}")
    # You might want to debug your objective function

[I 2026-01-02 16:13:38,433] A new study created in memory with name: xgboost_study_cpu


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2026-01-02 16:13:42,678] Trial 0 finished with value: 0.7173277780266298 and parameters: {'max_depth': 3, 'learning_rate': 0.07569822614567863, 'n_estimators': 670, 'subsample': 0.6502089551614696, 'colsample_bytree': 0.9226302920374178, 'min_child_weight': 2, 'gamma': 2.7928280234197396}. Best is trial 0 with value: 0.7173277780266298.
[I 2026-01-02 16:13:45,553] Trial 1 finished with value: 0.7187871428750948 and parameters: {'max_depth': 4, 'learning_rate': 0.07047243654157946, 'n_estimators': 484, 'subsample': 0.844902700926369, 'colsample_bytree': 0.5378355558317127, 'min_child_weight': 4, 'gamma': 1.649344158590504}. Best is trial 1 with value: 0.7187871428750948.
[I 2026-01-02 16:13:47,580] Trial 2 finished with value: 0.7075221586175533 and parameters: {'max_depth': 8, 'learning_rate': 0.012772144199567912, 'n_estimators': 147, 'subsample': 0.7100101749233771, 'colsample_bytree': 0.7726290677783649, 'min_child_weight': 4, 'gamma': 1.7318906479715979}. Best is trial 1 with va

In [56]:
classifier = xgboost.XGBClassifier(
    max_depth=3,  # Removed quotes and changed to parameter=value format
    learning_rate=0.08645441605311319, 
    n_estimators=915,
    subsample=0.9431912589210164,
    colsample_bytree=0.9060924587469108,
    min_child_weight=2, 
    gamma=0.5286514182187841
)

In [60]:
from sklearn.model_selection import cross_val_score

roc_scores = cross_val_score(
    classifier,
    x, y,
    cv=10,
    scoring='roc_auc'
)

print(roc_scores)
print("Mean ROC-AUC:", roc_scores.mean())


[0.72570665 0.72660811 0.72714995 0.72488845 0.72837594 0.72406989
 0.72695772 0.72788161 0.72639334 0.71522762]
Mean ROC-AUC: 0.7253259288948386


In [61]:
classifier.fit(x_train, y_train)   # train on FULL dataset

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9060924587469108
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [62]:
classifier.predict_proba(x_test)

array([[0.43632555, 0.56367445],
       [0.23007858, 0.7699214 ],
       [0.41218585, 0.58781415],
       ...,
       [0.52939606, 0.47060397],
       [0.5001847 , 0.4998153 ],
       [0.43189663, 0.5681034 ]], shape=(140000, 2), dtype=float32)

In [63]:
y_proba = classifier.predict_proba(x_test)[:,1]  # probability of class 1
roc_auc = roc_auc_score(y_test, y_proba)
print("Test ROC-AUC:", roc_auc)

Test ROC-AUC: 0.7255531849232277


In [64]:
model = xgb.XGBClassifier(device = 'cuda')

In [69]:


# Defining the objective function
def objective_gpu(trial):
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
    }

    model = xgb.XGBClassifier(**param, device="cuda")

    score = cross_val_score(model, x, y, cv=3, scoring="roc_auc").mean()
    return score

In [70]:
study = optuna.create_study(study_name="xgboost_study_cuda", direction="maximize")
study.optimize(objective_gpu, n_trials=10, show_progress_bar=True, n_jobs=-1)

# Retrieve the best parameter values
best_params = study.best_params
print(f"\nBest parameters: {best_params}")

[I 2026-01-02 16:39:11,763] A new study created in memory with name: xgboost_study_cuda


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2026-01-02 16:40:57,297] Trial 0 finished with value: 0.7118831281525683 and parameters: {'max_depth': 3, 'learning_rate': 0.04071025155539013, 'n_estimators': 369, 'subsample': 0.553488826168297, 'colsample_bytree': 0.8050962312587406, 'min_child_weight': 5, 'gamma': 0.5299103231002189}. Best is trial 0 with value: 0.7118831281525683.
[I 2026-01-02 16:41:07,531] Trial 5 finished with value: 0.7115238978209408 and parameters: {'max_depth': 5, 'learning_rate': 0.0154663382768483, 'n_estimators': 386, 'subsample': 0.8503468346155383, 'colsample_bytree': 0.7348150099944988, 'min_child_weight': 5, 'gamma': 3.4818135575840365}. Best is trial 0 with value: 0.7118831281525683.
[I 2026-01-02 16:41:16,266] Trial 7 finished with value: 0.7248128096089492 and parameters: {'max_depth': 5, 'learning_rate': 0.076461808783582, 'n_estimators': 469, 'subsample': 0.9717810731469305, 'colsample_bytree': 0.5371456958784091, 'min_child_weight': 9, 'gamma': 0.7689334752908972}. Best is trial 7 with value

In [71]:
study = optuna.create_study(study_name="xgboost_study_cuda", direction="maximize")
study.optimize(objective_gpu, n_trials=100, show_progress_bar=True, n_jobs=-1)

# Retrieve the best parameter values
best_params = study.best_params
print(f"\nBest parameters: {best_params}")

[I 2026-01-02 16:42:22,213] A new study created in memory with name: xgboost_study_cuda


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2026-01-02 16:42:50,468] Trial 1 finished with value: 0.7195832411857076 and parameters: {'max_depth': 6, 'learning_rate': 0.0777926604395986, 'n_estimators': 116, 'subsample': 0.6119419307353036, 'colsample_bytree': 0.531933265412, 'min_child_weight': 5, 'gamma': 2.37514745417435}. Best is trial 1 with value: 0.7195832411857076.
[I 2026-01-02 16:42:58,111] Trial 11 finished with value: 0.719154408099779 and parameters: {'max_depth': 7, 'learning_rate': 0.06964684713410037, 'n_estimators': 116, 'subsample': 0.6082016234629319, 'colsample_bytree': 0.8880372481547746, 'min_child_weight': 9, 'gamma': 0.5441152884201866}. Best is trial 1 with value: 0.7195832411857076.
[I 2026-01-02 16:43:56,491] Trial 12 finished with value: 0.7252461385975336 and parameters: {'max_depth': 5, 'learning_rate': 0.08127962877755603, 'n_estimators': 508, 'subsample': 0.967375522280508, 'colsample_bytree': 0.521997669627738, 'min_child_weight': 7, 'gamma': 0.2705481957057626}. Best is trial 12 with value: 0

In [74]:
classifier = xgboost.XGBClassifier(
  max_depth= 4,
    learning_rate= 0.08907067334125408,
    n_estimators= 917, 
    subsample= 0.784799836274861,
    colsample_bytree= 0.5632962690491155, 
    min_child_weight= 8, 
    gamma= 0.4415903019619828
)

In [75]:
from sklearn.model_selection import cross_val_score

roc_scores = cross_val_score(
    classifier,
    x, y,
    cv=10,
    scoring='roc_auc'
)

print(roc_scores)
print("Mean ROC-AUC:", roc_scores.mean())


[0.72694247 0.72846265 0.72827025 0.72601065 0.72962232 0.72526954
 0.72845867 0.72894561 0.72773188 0.71643629]
Mean ROC-AUC: 0.7266150345789735


In [76]:
classifier.fit(x_train, y_train)   # train on FULL dataset
classifier.predict_proba(x_test)

array([[0.43227953, 0.5677205 ],
       [0.23256552, 0.7674345 ],
       [0.43881333, 0.5611867 ],
       ...,
       [0.52638745, 0.47361258],
       [0.5010265 , 0.49897346],
       [0.4120133 , 0.5879867 ]], shape=(140000, 2), dtype=float32)

In [77]:
y_proba = classifier.predict_proba(x_test)[:,1]  # probability of class 1
roc_auc = roc_auc_score(y_test, y_proba)
print("Test ROC-AUC:", roc_auc)

Test ROC-AUC: 0.7269213062863717
